In [7]:
import os
import cv2
import numpy as np
import pandas as pd
from mtcnn.mtcnn import MTCNN
from sklearn.preprocessing import LabelEncoder
import joblib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical, normalize


IMG_SIZE = 50
BATCH_SIZE = 32
EPOCHS = 30
combined_dataset_dir = "combined_dataset"
webcam_dataset_dir = "dataset"
csv_path = "data/face/train.csv"
images_file_path = "data/face/Medical mask/Medical mask/Medical mask/images/"

print("🔄 Converting CSV-based dataset into folder structure...")
df = pd.read_csv(csv_path)
df = df[df['classname'].isin(["face_with_mask", "face_no_mask"])].reset_index(drop=True)

for index, row in df.iterrows():
    name, x1, x2, y1, y2, classname = row.values
    full_path = os.path.join(images_file_path, name)
    img_array = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
    if img_array is None:
        continue
    crop_img = img_array[int(x2):int(y2), int(x1):int(y1)]
    if crop_img is None or crop_img.size == 0:
        continue
    resized_img = cv2.resize(crop_img, (IMG_SIZE, IMG_SIZE))
    save_path = os.path.join(combined_dataset_dir, classname)
    os.makedirs(save_path, exist_ok=True)
    filename = f"{classname}_csv_{index}.png"
    cv2.imwrite(os.path.join(save_path, filename), resized_img)

for label in os.listdir(webcam_dataset_dir):
    source_folder = os.path.join(webcam_dataset_dir, label)
    target_folder = os.path.join(combined_dataset_dir, label)
    os.makedirs(target_folder, exist_ok=True)
    for i, file in enumerate(os.listdir(source_folder)):
        src = os.path.join(source_folder, file)
        dst = os.path.join(target_folder, f"{label}_webcam_{i}.png")
        img = cv2.imread(src, cv2.IMREAD_GRAYSCALE)
        if img is None or img.shape != (IMG_SIZE, IMG_SIZE):
            continue
        cv2.imwrite(dst, img)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    combined_dataset_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset='training'
)

val_gen = datagen.flow_from_directory(
    combined_dataset_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset='validation'
)

lbl = LabelEncoder()
lbl.fit(list(train_gen.class_indices.keys()))
joblib.dump(lbl, "label_encoder.pkl")

model = Sequential()
model.add(Conv2D(100, kernel_size=(3, 3), strides=2, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(train_gen.num_classes, activation='softmax'))

model.compile(
    optimizer=Adam(learning_rate=1e-3, decay=1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS)

model.save("mask_classifier.h5")


🔄 Converting CSV-based dataset into folder structure...
📥 Merging webcam dataset...
📊 Preparing training pipeline...
Found 4719 images belonging to 3 classes.
Found 1178 images belonging to 3 classes.
✅ Label encoder saved as label_encoder.pkl
🧠 Building model...
🚀 Training model...
Epoch 1/30


C:\Users\goura\anaconda3\envs\tfenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\goura\anaconda3\envs\tfenv\lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
C:\Users\goura\anaconda3\envs\tfenv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.6883 - loss: 0.6817 - val_accuracy: 0.7199 - val_loss: 0.5356
Epoch 2/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.7310 - loss: 0.5247 - val_accuracy: 0.7954 - val_loss: 0.4338
Epoch 3/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.7803 - loss: 0.4392 - val_accuracy: 0.8014 - val_loss: 0.4174
Epoch 4/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8215 - loss: 0.3928 - val_accuracy: 0.8311 - val_loss: 0.3754
Epoch 5/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8420 - loss: 0.3594 - val_accuracy: 0.8183 - val_loss: 0.3809
Epoch 6/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8604 - loss: 0.3312 - val_accuracy: 0.8285 - val_loss: 0.3738
Epoch 7/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8749 - loss: 0.2940 - val_accuracy: 0.8396 - val_loss: 0.3629
Epoch 8/30
148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.8823 - loss: 0.2883 - val_accuracy: 0.856

✅ Model saved as mask_classifier.h5


In [11]:
model = load_model("mask_classifier.h5")
lbl = joblib.load("label_encoder.pkl")

cap = cv2.VideoCapture(0)
detector = MTCNN()
img_size = 50

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector.detect_faces(rgb_frame)

    for face in faces:
        if face['confidence'] < 0.95:
            continue

        x, y, width, height = face['box']
        x, y = max(0, x), max(0, y)
        x2 = min(x + width, gray_frame.shape[1])
        y2 = min(y + height, gray_frame.shape[0])
        face_img = gray_frame[y:y2, x:x2]

        if face_img.size == 0 or face_img.shape[0] < 30 or face_img.shape[1] < 30:
            continue

        resized_face = cv2.resize(face_img, (img_size, img_size))
        normalized_face = resized_face.reshape(-1, img_size, img_size, 1)
        normalized_face = normalize(normalized_face, axis=1)

        prediction = model.predict(normalized_face, verbose=0)
        class_idx = np.argmax(prediction)
        confidence = prediction[0][class_idx]
        label = lbl.inverse_transform([class_idx])[0]

        if confidence < 0.80:
            label = "uncertain"

        # Color coding
        if label == "face_with_mask":
            color = (0, 255, 0)
        elif label == "face_no_mask":
            color = (0, 0, 255)
        else:
            color = (0, 255, 255)
        cv2.rectangle(frame, (x, y), (x2, y2), color, 2)
        text = f"{label} ({confidence * 100:.1f}%)"
        cv2.putText(frame, text, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Face Mask Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()